In [1]:
import pickle
import os
import re, string, unicodedata
import nltk
!pip install contractions
!pip install inflect
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

     |████████████████████████████████| 327kB 7.7MB/s 
     |████████████████████████████████| 266kB 14.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.1-cp37-cp37m-linux_x86_64.whl size=85256 sha256=23af447087439c1a7e127ea5dc480da0b9d10aa42c764ad2428c18e427b6592b
  Stored in directory: /root/.cache/pip/wheels/e4/ab/f7/cb39270df8f6126f3dd4c33d302357167086db460968cfc80c
Successfully built pyahocorasick
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
def DoctoDict():
  file_Names = os.listdir("/content/drive/MyDrive/IR_ASSIGNMENT_1/stories1")
  file_Paths = []
  for i in range(len(file_Names)):
    file_Paths.append("/content/drive/MyDrive/IR_ASSIGNMENT_1/stories1/"+file_Names[i])
  corpus = {}
  for i in range(len(file_Paths)):
    with open(file_Paths[i],encoding = "latin-1") as f_input:
      corpus[file_Names[i]] = [f_input.read()]
  return corpus

In [3]:
CORPUS = DoctoDict()
with open('CORPUS.pickle', 'wb') as handle:
  pickle.dump(CORPUS, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
with open('/content/CORPUS.pickle', 'rb') as handle:
  corpus = pickle.load(handle)

len(corpus.keys())

467

In [5]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    text=re.sub('\n',' ',text)
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace_contractions(text):
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def get_wordnet_pos(tag):
    tag_dict = {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
    return tag_dict.get(tag,wordnet.NOUN)

def lemmatize(words):
    """Lemmatize words in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    posTagged = nltk.pos_tag(words)
    wordnetTagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1][0])), posTagged))
    for word,tag in wordnetTagged:
      lemma = lemmatizer.lemmatize(word,tag)
      lemmas.append(lemma)
    return lemmas

def preProcess_html(fileName):
    sample = denoise_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize(words)
    return words  

def clean_text(text):
    # text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z0-9A-Z]',' ',text)
    text=re.sub(' +',' ',text)
    return text

def preProcessotherfiles(fileName):
    sample = clean_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize(words)
    return words 


In [6]:
for i in corpus.keys():
  if i.endswith(".html") or i.endswith(".header"):
    corpus[i][0] = preProcess_html(corpus[i][0])
  else:
    corpus[i][0] = preProcessotherfiles(corpus[i][0])

In [7]:
len(corpus.keys())

467

In [8]:
with open('DocTerms.pickle', 'wb') as handle:
  pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('/content/DocTerms.pickle', 'rb') as handle:
  #  docT = pickle.load(handle)

In [17]:
docT.keys()

dict_keys(['abyss.txt', '6ablemen.txt', '5orange.txt', '4moons.txt', '3wishes.txt', '3student.txt', '3lpigs.txt', '3gables.txt', '20.lws', '3sonnets.vrs', '19.lws', '14.lws', '17.lws', '18.lws', '16.lws', '100west.txt', '13chil.txt', 'bookem2', 'blackrdr', 'blossom.pom', 'blak', 'blh.txt', 'blasters.fic', 'blind.txt', 'blue', 'bookem.1', 'blabnove.txt', 'bluebrd.txt', 'blackp.txt', 'berternie.txt', 'bgb.txt', 'bgcspoof.txt', 'blabnove.hum', 'bishop00.txt', 'bestwish', 'beyond.hum', 'bern', 'beggars.txt', 'beast.asc', 'beautbst.txt', 'bigred.hum', 'assorted.txt', 'arctic.txt', 'batlslau.txt', 'bagelman.txt', 'angry_ca.txt', 'antcrick.txt', 'aquith.txt', 'archive', 'asop', 'arcadia.sty', 'aesop11.txt', 'alissadl.txt', 'aesopa10.txt', 'bagel.man', 'adv_alad.txt', 'alad10.txt', 'aircon.txt', 'aislesix.txt', 'aminegg.txt', 'angelfur.hum', 'aluminum.hum', 'abbey.txt', '7voysinb.txt', 'ab40thv.txt', '7oldsamr.txt', '6napolen.txt', 'aisle.six', 'altside.hum', 'advtthum.txt', 'advsayed.txt', 'a